<h1><center>Welcome to the Preload Automation Tool.</center></h1>

## Insert the Build Plan Path Below

In [86]:
from __future__ import division
import ipywidgets as widgets
from IPython.display import display
import os
output = widgets.Text(placeholder="0", layout=widgets.Layout(width="200px"),  disabled=True)

build_plan_path = widgets.Text(
    value= r"C:\Users\rs623u\automation\testruns\Automation_Build_Plan_AIC_DPA2A_CTS02_v1.8.xlsx", #change
    placeholder="Paste Build Plan Path",
    description="PATH1:",
    disabled=False
)
display(build_plan_path)

Text(value='C:\\Users\\rs623u\\automation\\testruns\\Automation_Build_Plan_AIC_DPA2A_CTS02_v1.8.xlsx', descrip…

In [87]:
button = widgets.Button(description="Update Path")
display(button)
def work1(build_plan_path):
    build_plan_path = str(build_plan_path)
    a, b = build_plan_path.find("'"), build_plan_path.find(",")
    build_plan_path = build_plan_path[a:b]
    build_plan_path = build_plan_path.replace("'", "")
    build_plan_path = build_plan_path
    print(build_plan_path)

def on_button_clicked(b):
    work1(build_plan_path)
button.on_click(on_button_clicked)

Button(description='Print Path', style=ButtonStyle())

## Insert the Preload Templates Folder Path Below

In [88]:
#paths_upload = widgets.FileUpload(multiple=False)
#display(paths_upload)
paths = widgets.Text(
    value= r"C:\\Users\rs623u\\automation\\testruns\testin\\",
    placeholder="Paste Preload Templates Path",
    description="PATH2",
    disabled=False
)
display(paths)

Text(value='C:\\\\Users\\rs623u\\\\automation\\\\testruns\\testin\\\\', description='PATH2', placeholder='Past…

In [89]:
button = widgets.Button(description="Update Path")
display(button)
def work2(paths):
    paths = str(paths)
    a, b = paths.find("'"), paths.find(",")
    paths = paths[a:b]
    paths = paths.replace("'", "")
    paths = paths
    print(paths)

def on_button_clicked(b):
    work2(paths)
button.on_click(on_button_clicked)

Button(description='Print Path', style=ButtonStyle())

## Insert the Output Templates Path Below

In [90]:
dest_folder = widgets.Text(
    value= r"C:\\Users\rs623u\\automation\\testruns\testout\\",
    placeholder="Paste Output Folder Path.",
    description="PATH3",
    disabled=False
)
display(dest_folder)

Text(value='C:\\\\Users\\rs623u\\\\automation\\\\testruns\\testout\\\\', description='PATH3', placeholder='Pas…

In [91]:
button = widgets.Button(description="Update Path")
display(button)
def work3(dest_folder):
    dest_folder = str(dest_folder)
    a, b = dest_folder.find("'"), dest_folder.find(",")
    dest_folder = dest_folder[a:b]
    dest_folder = dest_folder.replace("'", "")
    dest_folder = dest_folder
    print(dest_folder)


def on_button_clicked(b):
    work3(dest_folder)
button.on_click(on_button_clicked)

Button(description='Print Path', style=ButtonStyle())

In [92]:
preload_list = []
paths = str(paths.value)
build_plan_path = str(build_plan_path.value)
dest_folder = str(dest_folder.value)
for idx, item in enumerate(os.listdir(paths)):
    preload_list.append(paths+item) 



In [93]:
import os
import xlrd
import xlwings as xw

button = widgets.Button(description="Click Me!")
display(button)

# def check_automation_template(build_plan_path):
## check to see the ip's and oam protected colums in vm layout
## make sure vm-name in vm-layout matches the vnf name prefix in vnf-specs
## make sure set of vm-type matches set of vm-types in vnf specs
def work4(preload_list, build_plan_path):
    def calculate_vm_count(build_plan_path):
        """
        This function:
            - gathers the "# of VM's" per VM type for file generation 
            - creates list of properly named titles for final output folder based off of each vm-types vf-module-name
            - used for function calls at the end of the program
        """
        global final_vf_module_name
        global title_list

        pt = xw.Book(preload_path)
        vm_type = pt.sheets[4].range('B7').value

        # set variable to hold vm count
        global vm_count 
        # open workbook and specify which sheet you would like to access
        wb = xlrd.open_workbook(build_plan_path)
        sheet_names = wb.sheet_names()
        bp = wb.sheet_by_name(u'VNF-Specs')

        # string search VNF-Specs column headers and assign each columns reference position (int) to a variable
        # this avoids hard coding the position of certain columns
        # order does not matter
        for i in range(bp.nrows):
            for j in range(bp.ncols):
                if bp.cell_value(i, j) == "vm-type":
                    col_ref_vmt = j
                if bp.cell_value(i, j) == "# of VM's":
                    col_ref_vmc = j
                if bp.cell_value(i, j) == "vf-module-name":
                        col_ref_vfmn = j

        # Find the VMs that are on the build plan
        list_of_vms = []
        for i in range(bp.nrows):
            list_of_vms.append(bp.cell_value(i, col_ref_vmt))
        # remove all empty stings
        list_of_vms = list(filter(None, list_of_vms))
        # access the vm_type for each file
        pt = xw.Book(preload_path)
        vm_type_check = pt.sheets[4].range('B7').value
        # if the vm_type is not on the build plan just build an empty list and do nothing
        if vm_type_check not in list_of_vms:
            title_list = []
        # else follow through with
        else:
            # create dict of vm-type and each vm-types VM Count
            count_dict = {}
            for i in range(bp.nrows):
                vm = bp.cell_value(i, col_ref_vmt)
                count = bp.cell_value(i, col_ref_vmc)
                count_dict[vm] = count

            # cast as integer as it is pulled as a string
            for k, v in count_dict.items():
                if k == vm_type:
                    vm_count = int(v)

            # create dictionary of vm-names and vfmn for title generation  
            vfmn_dict = {}
            for i in range(bp.nrows):
                vm = bp.cell_value(i, col_ref_vmt)
                vfmn = bp.cell_value(i, col_ref_vfmn)
                vfmn_dict[vm] = vfmn

            # give the titles based on which vm-type the current file is
            title_list = []
            for k, v in vfmn_dict.items():
                for i in range(1, vm_count+1):
                    if k != '' and k == vm_type:
                        title_list.append(v)

    def change_tag(preload_path, build_plan_path):
        """
        This function:
            - initiates changes for all tag values
        """
        # open workbook and specify which sheet you would like to access
        wb = xlrd.open_workbook(build_plan_path)
        sheet_names = wb.sheet_names()
        bp = wb.sheet_by_name(u'Common Parameters')

        params = []
        for i in range(0, bp.nrows):
            if bp.cell_value(i, 0) == "Parameter Name":
                params.append(i)
        start = int(params[-1]) + 1

        # create dict of parameter name and associated value
        tag_dict = {}
        for i in range(start, bp.nrows):
            par = bp.cell_value(i, 0)
            val = bp.cell_value(i, 1)
            tag_dict[par] = val

        # open workbook and specify which sheet you would like to access
        wb = xlrd.open_workbook(preload_path)
        sheet_names = wb.sheet_names()
        tag_sheet = wb.sheet_by_name(u'Tag-values')

        # implement changes to template
        for k, v in tag_dict.items():
            for i in range(tag_sheet.nrows):
                if(tag_sheet.cell_value(i, 1) == k and k != ''):
                    wb = xw.Book(preload_path)
                    wb.sheets[8].range('C' + str(i+1)).value = v
                    
    for preload_path in preload_list:
        if "base" not in preload_path:
            calculate_vm_count(build_plan_path)
            for titles in range(0, len(title_list)):
                wb = xw.Book(preload_path)
                suffix = "xlsm"
                change_tag(preload_path, build_plan_path)     
                print(dest_folder + title_list[titles] + str(titles+1) + ".xlsm")
                #final = os.path.join(dest_folder, title_list[titles] + str(titles+1)+ "." + suffix)
                #print("FINAL: ", final)
                #wb.save(final)
                wb.save(dest_folder + title_list[titles] + str(titles+1) + ".xlsm")
                wb.close()

def on_button_clicked(b):
    work4(preload_list, build_plan_path)
button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arqrtim02_qrt_1.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arqrtim02_qrt_2.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arvlbct02_vlb_1.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arlbact02_lba_1.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arprbct02_prb_1.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arprbct02_prb_2.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arprbct02_prb_3.xlsm
C:\\Users\rs623u\\automation\\testruns\testout\\zdpa2arprbct02_prb_4.xlsm
